#### DATA CLEANING

Code to perform data wrangling. The code reads data from mysql, namely the bom, wiki, omdb and comb_index (the output file from jaro-winkler) tables. 

Outputfile: Movies that contain budget and gross information

Note: The movie should be present in two sources. The movies with budget/gross information that are missed in this section are added before the data analysis.

Authors: nelsonds@uw.edu, gmaria@uw.edu, sseth12@uw.edu

In [1]:
# Load libraries and instatiate DB connection

%matplotlib inline
import pandas as pd
import pymysql
import numpy as np
import time
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.max_seq_items = 2000

In [2]:
# Connect to the database
connection = pymysql.connect(host='localhost', user='root', password='mprophet', db='movies')

# Pull table from database
cur = connection.cursor()

cur.execute("select * FROM bom")
bom_sql = []
for row in cur: bom_sql.append(list(row))

cur.execute("select * FROM wiki")
wiki_sql = []
for row in cur: wiki_sql.append(list(row))
    
cur.execute("select * FROM omdb")
omdb_sql = []
for row in cur: omdb_sql.append(list(row))
    
cur.execute("select * FROM comb_index")
comb_sql = []
for row in cur: comb_sql.append(list(row))
    
#cur.execute("select * FROM movies_final")
#movie_sql = []
#for row in cur: movie_sql.append(list(row))
    
cur.close()
connection.close()

##### Reading data from db

In [3]:
# Store bom table information in DataFrame

bom_columns =         ['bom_title', 'bom_link', 'key_bom', 'bom_date', 'bom_year', 'bom_name', 'bom_budget',
                       'bom_domestic_gross', 'bom_lifetime_gross', 'bom_release_date', 'bom_runtime', 'bom_distributor',
                       'bom_genre', 'bom_rating', 'bom_awards', 'bom_actor', 'bom_assoc_producer', 'bom_cinematographer',
                       'bom_composer', 'bom_director', 'bom_exec_producer', 'bom_line_producer', 'bom_players', 'bom_producer',
                       'bom_writer', 'bom_close_date', 'bom_domestic_summary', 'bom_in_release', 'bom_limited_opening_weekend',
                       'bom_opening_weekend', 'bom_wide_opening_weekend', 'bom_widest_release', 'bom_domestic_revenue',
                       'bom_foreign_revenue', 'bom_worldwide_revenue', 'bom_total_lifetime_gross']

bom = pd.DataFrame(bom_sql, columns=bom_columns)
#bom.to_csv('bom.csv')
print('Total movies from Box Office Mojo:', len(bom))
bom.head(1)

Total movies from Box Office Mojo: 16826


,bom_title,bom_link,key_bom,bom_date,bom_year,bom_name,bom_budget,bom_domestic_gross,bom_lifetime_gross,bom_release_date,bom_runtime,bom_distributor,bom_genre,bom_rating,bom_awards,bom_actor,bom_assoc_producer,bom_cinematographer,bom_composer,bom_director,bom_exec_producer,bom_line_producer,bom_players,bom_producer,bom_writer,bom_close_date,bom_domestic_summary,bom_in_release,bom_limited_opening_weekend,bom_opening_weekend,bom_wide_opening_weekend,bom_widest_release,bom_domestic_revenue,bom_foreign_revenue,bom_worldwide_revenue,bom_total_lifetime_gross
0,#horror,http://www.boxofficemojo.com/movies/?id=horrorifc.htm,b0,20-Nov-15,2015\r,#Horror,N/A,,,20-Nov-15,1 hrs. 30 min.,IFC,Horror,Unknown,No Academy Awards Information\r,"""['Balthazar Getty', 'Timothy Hutton', 'Natasha Lyonne', 'Taryn Manning', 'Chloe Sevigny']""",,,,,,,,,\r,,No Domestic Summary Information,,,,,\r,,,,No Total Lifetime Gross Information\r


In [4]:
wiki_columns = ['wiki_budget',
                "wiki_cinematography","wiki_director","wiki_distributor", 
                "wiki_editing", "wiki_gross" , "wiki_music", 
                "wiki_pageid", "wiki_producer", "wiki_released", 
                "wiki_runtime", "wiki_screenplay", "wiki_starring", 
                "wiki_studio","wiki_title", "key_wiki", "year_wiki",
                "year_wiki_format"];

wiki = pd.DataFrame(wiki_sql, columns=wiki_columns)
#bom.to_csv('bom.csv')
print('Total movies from Wikipedia:', len(wiki))

wiki['key_wiki'] = [str(v).lstrip() for v in wiki['key_wiki']]
wiki['key_wiki']= [str(v).rstrip() for v in wiki['key_wiki']]

wiki.head(1)

Total movies from Wikipedia: 40981


,wiki_budget,wiki_cinematography,wiki_director,wiki_distributor,wiki_editing,wiki_gross,wiki_music,wiki_pageid,wiki_producer,wiki_released,wiki_runtime,wiki_screenplay,wiki_starring,wiki_studio,wiki_title,key_wiki,year_wiki,year_wiki_format
0,,"""[Gilbert Salas, Heather Rae]""",[Heather Rae],[],"""[Gregory Bayne, Heather Rae]""",,[John Trudell & Bad Dog],5975700,"""[Heather Rae, Elyse Katz]""","""[2005, 01, 20, Sundance Film Festival, 2006, 02, 24, United States]""",[80 minutes],,[],,trudell,w39443,None,None


In [5]:
omdb_columns = ["omdb_movie_id", "omdb_movie_year", "omdb_title", "omdb_rated", "omdb_response", 
                 "omdb_language" , "omdb_country", "omdb_metascore", "omdb_imdbrating", "omdb_released",
                 "omdb_runtime", "omdb_type", "omdb_poster", "omdb_imdbvotes", "omdb_awards", "omdb_genres",
                 "omdb_actors", "omdb_directors","omdb_writers","omdb_plot","key_omdb"]

omdb = pd.DataFrame(omdb_sql, columns=omdb_columns)
#bom.to_csv('bom.csv')
print('Total movies from Omdb:', len(omdb))

omdb.head(1)

Total movies from Omdb: 23254


,omdb_movie_id,omdb_movie_year,omdb_title,omdb_rated,omdb_response,omdb_language,omdb_country,omdb_metascore,omdb_imdbrating,omdb_released,omdb_runtime,omdb_type,omdb_poster,omdb_imdbvotes,omdb_awards,omdb_genres,omdb_actors,omdb_directors,omdb_writers,omdb_plot,key_omdb
0,1,1933,A Bedtime Story,N/A,None,"English, French",USA,N/A,None,22 Apr 1933,87 min,movie,https://images-na.ssl-images-amazon.com/images/M/MV5BODhjZWY3ZjQtN2RjNy00ZTE4LTgzMmQtYmEyMzc0N2M...,None,N/A,Musical|Romance|Comedy,Maurice Chevalier|Helen Twelvetrees|Edward Everett Horton|Adrienne Ames,Norman Taurog,Roy Horniman (novel)|Benjamin Glazer (adaptation)|Nunnally Johnson (screenplay)|Waldemar Young (...,Parisian playboy plays father to an abandoned baby who interferes with his womanizing.,o0


In [6]:
comb_columns = ["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title","year_bom",
               "year_bom_format","year_wiki", "year_wiki_format", "year_omdb", "year_omdb_format","score"]
comb_index = pd.DataFrame(comb_sql, columns=comb_columns)
#bom.to_csv('bom.csv')
print('Total movies in combined file:', len(comb_index))
comb_index.head()

Total movies in combined file: 24665


,key_bom,key_wiki,key_omdb,bom_title,wiki_title,omdb_title,year_bom,year_bom_format,year_wiki,year_wiki_format,year_omdb,year_omdb_format,score
0,b15584,w39443,o18495,trudell,trudell,trudell,3-Feb-06,2006,2005,2005,20-Jan-05,2005,1\r
1,b14216,w28904,o12944,the party at kitty and stud's,the party at kitty and stud's,the party at kitty and stud's,6-Jul-76,1976,1970,1970,10-Feb-70,1970,1\r
2,,w4957,o17832,,borrowed wives,borrowed wives,,,20-Aug,20-Aug,20-Aug-30,1930,1\r
3,,w40302,o15689,,the vagabond trail,the vagabond trail,,,9-Mar,9-Mar,9-Mar-24,1924,1\r
4,,w11443,o9690,,eddie's million dollar cook-off,eddie's million dollar cook-off,,,2003,2003,18-Jul-03,2003,1\r


##### Creating the combined data (to be loaded into the table movies_final)

* Remove duplicate rows from wiki
* Remove duplicate indexes frim comb_index
* Fetching movies with only budget and gross information
      > For bom, fetch movie if either bom_domestic_gross or bom_worldwide_revenue information is available

In [7]:
# Removing duplicate rows from wiki
wiki = wiki.fillna("")
print(wiki['key_wiki'].unique().shape)
print(wiki.shape)
wiki = wiki.drop_duplicates(['key_wiki'],keep ='last')
print(wiki.shape)

(40746,)
(40981, 18)
(40746, 18)


In [8]:
# CLEANING COMB_INDEX FOR DUPLICATES
print(comb_index.shape)

print('bom unique keys',comb_index['key_bom'].unique().shape)
a = comb_index[comb_index['key_bom']!=""]
print('bom not null',a.shape)

# print the rows with where bom ky is duplicated
k = a[a.duplicated(['key_bom'])]['key_bom'].tolist()
#comb_index[comb_index['key_bom'].isin(k)]

# deleting duplicate rows
comb_index = comb_index.drop(comb_index.index[[7626,16978,22685,23066,23068,23155,23975,24029,24036,24054,24093,
                                              24103,24120,24123,24174,24257,24268,24269]])

comb_index.reset_index(inplace = True,drop =True)
print('Shape, combined_index after dropping bom duplicates',comb_index.shape)

(24665, 13)
bom unique keys (7096,)
bom not null (7112, 13)
Shape, combined_index after dropping bom duplicates (24647, 13)


In [9]:
print('omdb unique keys',comb_index['key_omdb'].unique().shape)
a = comb_index[comb_index['key_omdb']!=""]
print('omdb not null',a.shape)

# print the rows with where bom ky is duplicated
k = a[a.duplicated(['key_omdb'])]['key_omdb'].tolist()
comb_index[comb_index['key_omdb'].isin(k)]

# deleting duplicate rows
comb_index = comb_index.drop(comb_index.index[[15696,21655]])

comb_index.reset_index(inplace = True,drop =True)
print('Shape, combined_index after dropping omdb duplicates',comb_index.shape)

omdb unique keys (21893,)
omdb not null (21894, 13)
Shape, combined_index after dropping omdb duplicates (24645, 13)


In [10]:
print('wiki unique keys',comb_index['key_wiki'].unique().shape)
a = comb_index[comb_index['key_wiki']!=""]
print('wiki not null',a.shape)

# print the rows with where bom ky is duplicated
k = a[a.duplicated(['key_wiki'])]['key_wiki'].tolist()
comb_index[comb_index['key_wiki'].isin(k)]

# deleting duplicate rows
comb_index = comb_index.drop(comb_index.index[[1895,22386,22420,23060,23091,23854,23901,23944,24000,24006,
                                              24016,24022,24025,24066,24090,24094,24171,24264,24299,24383,24402,
                                              24454,24619,24643,24289,24401]])

comb_index.reset_index(inplace = True,drop =True)
print('Shape, combined_index after dropping wiki duplicates',comb_index.shape)

wiki unique keys (24621,)
wiki not null (24645, 13)
Shape, combined_index after dropping wiki duplicates (24619, 13)


In [11]:
print(comb_index['key_bom'].unique().shape)
print(comb_index['key_wiki'].unique().shape)
print(comb_index['key_omdb'].unique().shape)

print(comb_index.groupby(['key_omdb','key_bom','key_wiki']).count().shape)

comb_index = comb_index.drop_duplicates(['key_omdb','key_bom','key_wiki'],keep='last')
print("COMBINED INDEX AFTER REMOVING DUPLICATE KEYS: ", comb_index.shape)

(7070,)
(24619,)
(21892,)
(24619, 10)
COMBINED INDEX AFTER REMOVING DUPLICATE KEYS:  (24619, 13)


In [12]:
# Filtering wiki for data for which wiki_budget and wiki_gross exists
print("no:of movies in bom where budget is not null:")
print(bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A")].shape)
bom['bom_budget'] = [str(v).lstrip() for v in bom['bom_budget']]
bom['bom_budget']= [str(v).rstrip() for v in bom['bom_budget']]

print("no:of movies in bom where budget and bom_domestic_gross is not null:")
f = bom.bom_domestic_gross
print(bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A") & (f!="")].shape)

print("no:of movies in bom where budget and bom_worldwide_revenue is not null:")
g = bom.bom_worldwide_revenue
print(bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A") & (g!="")].shape)

bom_f = bom[(bom.bom_budget!="") & (bom.bom_budget!="N/A")]
bom_f1 = bom_f[(f!="")|(g!="")]
bom_f1.reset_index()

bom_f1= bom_f1[["key_bom","bom_title","bom_budget","bom_domestic_gross","bom_worldwide_revenue"]]
print("Bom movies with budget/gross (domestic or worldwide) info: ", bom_f1.shape)

no:of movies in bom where budget is not null:
(3114, 36)
no:of movies in bom where budget and bom_domestic_gross is not null:
(3066, 36)
no:of movies in bom where budget and bom_worldwide_revenue is not null:
(2530, 36)
Bom movies with budget/gross (domestic or worldwide) info:  (3076, 5)


/home/ubuntu/anaconda3/lib/python3.5/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [13]:
# Filtering the rows and intersted columns
print("no:of movies in wiki where budget and gross revenue is not null:")
print(wiki[(wiki.wiki_budget!="") & (wiki.wiki_gross!="")].shape)

wiki_f = wiki[(wiki.wiki_budget!="") & (wiki.wiki_gross!="")]
wiki_f = wiki_f[["key_wiki","wiki_title","wiki_budget","wiki_gross"]]
print("Wiki movies with budget gross information", wiki_f.shape)


no:of movies in wiki where budget and gross revenue is not null:
(6935, 18)
Wiki movies with budget gross information (6935, 4)


In [14]:
#FETCHING KEYS FROM COMB_INDEX DF
comb_index_f = comb_index[["key_bom","key_wiki","key_omdb","omdb_title","year_bom_format",
                           "year_wiki_format","year_omdb_format"]]

wiki_f_comb = pd.merge(wiki_f,comb_index_f,on = "key_wiki",how="left")
print(wiki_f_comb.shape)
wiki_f_comb.head()

(6935, 10)


,key_wiki,wiki_title,wiki_budget,wiki_gross,key_bom,key_omdb,omdb_title,year_bom_format,year_wiki_format,year_omdb_format
0,w25445,the mod squad,[$50 million],"""[$15,415,361]""",b13998,,,1999,1999,
1,w24917,mickey (2004 film),[$6 million],"""[$294,758]""",NaN,NaN,NaN,NaN,NaN,NaN
2,w7258,a christmas story,[$3.2 million],[$19.3 million],b274,o22363,a christmas story,1983,1983,1983
3,w40380,the vanishing (1993 film),[$23 million],"""[$14,543,394]""",NaN,NaN,NaN,NaN,NaN,NaN
4,w21140,kundun,[$28],[$5.7],b7039,o6452,kundun,1997,1997,1997


In [15]:
wiki_bom_f_comb = pd.merge(bom_f,wiki_f_comb,on = "key_bom",how="outer")
print(wiki_bom_f_comb.shape)
wiki_bom_f_comb = wiki_bom_f_comb.fillna("")
wiki_bom_f_comb.head()

(8029, 45)


,bom_title,bom_link,key_bom,bom_date,bom_year,bom_name,bom_budget,bom_domestic_gross,bom_lifetime_gross,bom_release_date,bom_runtime,bom_distributor,bom_genre,bom_rating,bom_awards,bom_actor,bom_assoc_producer,bom_cinematographer,bom_composer,bom_director,bom_exec_producer,bom_line_producer,bom_players,bom_producer,bom_writer,bom_close_date,bom_domestic_summary,bom_in_release,bom_limited_opening_weekend,bom_opening_weekend,bom_wide_opening_weekend,bom_widest_release,bom_domestic_revenue,bom_foreign_revenue,bom_worldwide_revenue,bom_total_lifetime_gross,key_wiki,wiki_title,wiki_budget,wiki_gross,key_omdb,omdb_title,year_bom_format,year_wiki_format,year_omdb_format
0,(500) days of summer,http://www.boxofficemojo.com/movies/?id=500daysofsummer.htm,b10,17-Jul-09,2009\r,(500) Days of Summer,$7.5 million,32391374,,17-Jul-09,1 hrs. 35 min.,Fox Searchlight,Romance,PG-13,No Academy Awards Information\r,"""['Zooey Deschanel', 'Geoffrey Arend', 'Chloe Moretz', 'Clark Gregg', 'Minka Kelly*']""",,,"""['Mychael Danna', 'Rob Simonsen']""",['Marc Webb'],,,,"""['Mason Novick', 'Mark S. Waters']""","""['Scott Neustadter', 'Michael H. Weber']""\r",""" November 22, 2009""",,133 days / 19 weeks,"""<b> $834,501</b>""",,"""$3,739,702 """,""" 1,048 theaters""\r","""$32,391,374 ""","""$28,331,360 ""","""$60,722,734 """,\r,,,,,,,,,
1,10 things i hate about you,http://www.boxofficemojo.com/movies/?id=10thingsihateaboutyou.htm,b24,31-Mar-99,1999\r,10 Things I Hate About You,$30 million,38178166,,31-Mar-99,1 hrs. 37 min.,Buena Vista,Romantic Comedy,PG-13,No Academy Awards Information\r,"""['Julia Stiles', 'Heath Ledger']""",,,,,,,,"""['Jeffrey Chernov', 'Andrew Lazar']""","""['Karen McCullah Lutz', 'Kirsten Smith']""\r",,,,,"""$8,330,681 """,,""" 2,311 theaters""\r","""$38,178,166 ""","""$15,300,000 ""","""$53,478,166 """,\r,,,,,,,,,
2,"""10,000 b.c.""",http://www.boxofficemojo.com/movies/?id=10000bc.htm,b27,7-Mar-08,2008\r,"""10,000 B.C.""",$105 million,94784201,,7-Mar-08,1 hrs. 49 min.,Warner Bros.,Period Adventure,PG-13,No Academy Awards Information\r,"""['Steven Strait', 'Camilla Belle']""",,,"""['Harald Kloser', 'Thomas Wander']""",['Roland Emmerich'],,,,"""['Mark Gordon', 'Harald Kloser', 'Scott Mednick', 'Thomas Tull']""","""['Roland Emmerich', 'Harald Kloser']""\r",""" June 19, 2008""",,105 days / 15 weeks,,"""$35,867,488 """,,""" 3,454 theaters""\r","""$94,784,201 ""","""$175,000,000 ""","""$269,784,201 """,\r,w123,"""10,000 bc """,[$105 million],[$269.8 million],,,2008,2008,
3,102 dalmatians,http://www.boxofficemojo.com/movies/?id=102dalmatians.htm,b38,22-Nov-00,2000\r,102 Dalmatians,$85 million,66957026,,22-Nov-00,1 hrs. 44 min.,Buena Vista,Family Comedy,G,Nominated for One Oscar.\r,"""['Glenn Close', 'Gerard Depardieu', 'Ioan Gruffudd']""",,,,,,,,,\r,,,,,"""$19,883,351 """,,""" 2,704 theaters""\r","""$66,957,026 ""","""$116,654,745 ""","""$183,611,771 """,\r,w299,102 dalmatians,[$85 million],[$183.6 million],o10558,102 dalmatians,2000,2000,2000
4,10th &amp; wolf,http://www.boxofficemojo.com/movies/?id=10thandwolf.htm,b39,18-Aug-06,2006\r,10th &amp; Wolf,$8 million,54702,,18-Aug-06,1 hrs. 50 min.,ThinkFilm,Crime Drama,R,No Academy Awards Information\r,"""['James Marsden', 'Giovanni Ribisi', 'Brad Renfro', 'Piper Perabo', 'Dennis Hopper', 'Lesley An...",,,['Aaron Zigman'],,,,,,\r,""" September 14, 2006""",,28 days / 4 weeks,,"""$20,774 """,,6 theaters\r,"""$54,702 ""","""$88,749 ""","""$143,451 """,\r,,,,,,,,,


In [16]:
# Checking one last time for duplicate rows
print('wiki unique keys',wiki_bom_f_comb['key_wiki'].unique().shape)
a = wiki_bom_f_comb[wiki_bom_f_comb['key_wiki']!=""]
print('wiki not null',a.shape)

# print the rows with where bom ky is duplicated
k = a[a.duplicated(['key_wiki'])]['key_wiki'].tolist()
wiki_bom_f_comb[wiki_bom_f_comb['key_wiki'].isin(k)]

print(wiki_bom_f_comb.groupby(['key_omdb','key_bom','key_wiki']).count().shape)

wiki unique keys (6936,)
wiki not null (6935, 45)
(8029, 42)


In [17]:
# Writing to CSV
wiki_bom_f_comb = wiki_bom_f_comb[["key_bom","key_wiki","key_omdb","bom_title","wiki_title","omdb_title",
                                  "year_bom_format","year_wiki_format","year_omdb_format",
                                  "wiki_budget","wiki_gross","bom_budget","bom_domestic_gross","bom_worldwide_revenue"]]
wiki_bom_f_comb = wiki_bom_f_comb[wiki_bom_f_comb['key_wiki'].str.startswith('w')]
wiki_bom_f_comb = wiki_bom_f_comb[wiki_bom_f_comb['key_wiki']!="what now"]
wiki_bom_f_comb = wiki_bom_f_comb[wiki_bom_f_comb['key_wiki']!="winter of frozen dreams"]
wiki_bom_f_comb.reset_index(inplace=True,drop=True)
print(wiki_bom_f_comb.shape)
wiki_bom_f_comb.to_csv("CombinedFilev2.csv",index = None)
wiki_bom_f_comb.to_csv("combinedfilev2.txt",sep="\t",index = None)

(6843, 14)


In [18]:
wiki_bom_f_comb.shape

(6843, 14)

CombinedFilev2.csv was uploaded into mysql. Table name: movies_final. This table acts as the base data file on which the data analysis is performed